In [3]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import pickle
import numpy as np

# --- LANGKAH 0: PENGATURAN DAN FUNGSI BANTU ---
print("Mulai proses pembuatan model...")

# Ganti dengan kredensial database PostgreSQL Anda
DB_USER = "postgres"
DB_PASSWORD = "DBmiko"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "dbexam"

# Buat koneksi ke database
try:
    db_engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
    print("Koneksi ke database berhasil.")
except Exception as e:
    print(f"Koneksi gagal: {e}")
    exit()

def grade_to_point(grade):
    """Mengubah grade numerik (0-100) ke skala IP (0-4)."""
    if grade >= 85: return 4.0
    if grade >= 75: return 3.0
    if grade >= 65: return 2.0
    if grade >= 55: return 1.0
    return 0.0

def difficulty_to_numeric(level):
    """Mengubah tingkat kesulitan ke angka."""
    if level == 'Sulit': return 3
    if level == 'Sedang': return 2
    return 1

# --- LANGKAH 1: MENGAMBIL DATA MENTAH ---
try:
    enrollment_df = pd.read_sql("SELECT * FROM enrollment", db_engine)
    student_df = pd.read_sql("SELECT * FROM student", db_engine)
    course_df = pd.read_sql("SELECT * FROM course", db_engine)
    difficulty_df = pd.read_sql("SELECT * FROM course_difficulty", db_engine)
    attendance_df = pd.read_sql("SELECT * FROM attendance", db_engine)
    print("Langkah 1: Pengambilan data mentah dari database selesai.")
except Exception as e:
    print(f"Gagal mengambil data: {e}")
    exit()

# --- LANGKAH 2: REKAYASA FITUR (FEATURE ENGINEERING) ---
print("Langkah 2: Memulai rekayasa fitur...")

# Asumsi: Setiap mata kuliah bernilai 3 SKS
course_df['sks'] = 3

# Gabungkan data untuk mempermudah perhitungan
df_merged = enrollment_df.merge(student_df, on='stu_id')
df_merged = df_merged.merge(course_df, on='course_id')
df_merged = df_merged.merge(difficulty_df, on='course_id')
df_merged = df_merged.merge(attendance_df, on='enroll_id')

# Hitung IP per semester untuk setiap mahasiswa
df_merged['points'] = df_merged['grade'].apply(grade_to_point)
df_merged['quality_points'] = df_merged['points'] * df_merged['sks']
ip_per_semester = df_merged.groupby(['stu_id', 'semester_id']).agg(
    total_quality_points=('quality_points', 'sum'),
    total_sks=('sks', 'sum')
).reset_index()
ip_per_semester['ip_semester'] = ip_per_semester['total_quality_points'] / ip_per_semester['total_sks']

# Membuat dataset final untuk machine learning
prediction_instances = []

# Iterasi setiap record IP sebagai target prediksi
for index, row in ip_per_semester.iterrows():
    current_student_id = row['stu_id']
    current_semester_id = row['semester_id']
    
    # --- Target ---
    target_ip = row['ip_semester']

    # --- Hitung Fitur Historis ---
    previous_semesters_ip = ip_per_semester[
        (ip_per_semester['stu_id'] == current_student_id) &
        (ip_per_semester['semester_id'] < current_semester_id)
    ]
    
    if not previous_semesters_ip.empty:
        ipk_kumulatif = previous_semesters_ip['ip_semester'].mean()
        ip_semester_lalu = previous_semesters_ip[previous_semesters_ip['semester_id'] == current_semester_id - 1]['ip_semester'].values[0]
    else:
        # Kasus untuk semester pertama
        ipk_kumulatif = 0
        ip_semester_lalu = 0
        
    # --- Hitung Fitur Semester Ini (Sebagai Rencana Studi) ---
    courses_this_semester = df_merged[
        (df_merged['stu_id'] == current_student_id) &
        (df_merged['semester_id'] == current_semester_id)
    ]
    
    total_sks_semester_ini = courses_this_semester['sks'].sum()
    
    courses_this_semester['difficulty_numeric'] = courses_this_semester['difficulty_level'].apply(difficulty_to_numeric)
    avg_difficulty_score = np.average(courses_this_semester['difficulty_numeric'], weights=courses_this_semester['sks'])
    
    # Ambil data demografis
    student_info = student_df[student_df['stu_id'] == current_student_id]
    gender = student_info['gender'].values[0]
    departemen = student_info['dept_id'].values[0] # Menggunakan ID departemen
    
    prediction_instances.append({
        'ipk_kumulatif': ipk_kumulatif,
        'ip_semester_lalu': ip_semester_lalu,
        'total_sks_semester_depan': total_sks_semester_ini,
        'avg_difficulty_score': avg_difficulty_score,
        'departemen': departemen,
        'gender': gender,
        'IP_Semester_Depan': target_ip
    })

ml_df = pd.DataFrame(prediction_instances)
# Hapus baris dengan nilai NaN jika ada (misal, jika ada data yang tidak lengkap)
ml_df.dropna(inplace=True)
print("Langkah 2: Rekayasa fitur selesai. Dataset siap untuk model.")
print(ml_df.head())

# --- LANGKAH 3: MEMPERSIAPKAN DATA (PREPROCESSING) ---
print("Langkah 3: Memulai preprocessing data...")
df_processed = pd.get_dummies(ml_df, columns=['departemen', 'gender'], drop_first=True)

# Definisikan fitur dan target
features = [col for col in df_processed.columns if col != 'IP_Semester_Depan']
X = df_processed[features]
y = df_processed['IP_Semester_Depan']
print("Langkah 3: Preprocessing data selesai.")

# --- LANGKAH 4: MEMISAHKAN DATA (TRAIN-TEST SPLIT) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Langkah 4: Data telah dibagi menjadi data training dan testing.")

# --- LANGKAH 5: MELATIH MODEL (MODEL TRAINING) ---
model = LinearRegression()
model.fit(X_train, y_train)
print("Langkah 5: Model Regresi Linear berhasil dilatih.")

# --- LANGKAH 6: MENGEVALUASI MODEL ---
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n--- Hasil Evaluasi Model ---")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print("Artinya: Rata-rata, prediksi IP model ini meleset sekitar {:.4f} poin dari nilai sebenarnya.".format(mae))
print(f"R-squared (R²): {r2:.4f}")
print("Artinya: Sekitar {:.1f}% variasi data IP dapat dijelaskan oleh model ini.".format(r2 * 100))
print("--------------------------\n")

# --- LANGKAH 7: MENYIMPAN MODEL & KOLOM ---
model_filename = 'model_prediksi_ip.pkl'
columns_filename = 'model_columns.pkl'

with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

with open(columns_filename, 'wb') as file:
    pickle.dump(X.columns.tolist(), file)

print(f"Langkah 7: Model telah disimpan ke '{model_filename}'")
print(f"Nama kolom fitur telah disimpan ke '{columns_filename}'")
print("\nProses Selesai!")

Mulai proses pembuatan model...
Koneksi ke database berhasil.
Langkah 1: Pengambilan data mentah dari database selesai.
Langkah 2: Memulai rekayasa fitur...
Langkah 2: Rekayasa fitur selesai. Dataset siap untuk model.
   ipk_kumulatif  ip_semester_lalu  total_sks_semester_depan  \
0       0.000000          0.000000                        12   
1       1.500000          1.500000                         9   
2       2.083333          2.666667                         9   
3       0.000000          0.000000                         6   
4       0.500000          0.500000                         6   

   avg_difficulty_score  departemen  gender  IP_Semester_Depan  
0                   1.0           1  Female           1.500000  
1                   1.0           1  Female           2.666667  
2                   1.0           1  Female           1.333333  
3                   1.0           1  Female           0.500000  
4                   1.0           1  Female           2.000000  
Langkah